# imports

In [ ]:
# imports
from niftypet import nipet
from niftypet import nimpa
from os import path
%matplotlib notebook
import matplotlib.pyplot as plt

print(nimpa.gpuinfo())
# get all the scanner parameters
mMRpars = nipet.get_mmrparams()

# load

In [ ]:
folderin = "amyloidPET_FBP_TP0"
# automatically categorise the input data
datain = nipet.classify_input(folderin, mMRpars)

# output path
opth = path.join(datain['corepath'], 'niftyout')
# switch on verbose mode
mMRpars['Cnt']['VERBOSE'] = True

datain

In [ ]:
# obtain the hardware mu-map (the bed and the head&neck coil)
muhdct = nipet.hdw_mumap(datain, [1,2,4], mMRpars, outpath=opth, use_stored=True)

In [ ]:
# obtain the MR-based human mu-map
muodct = nipet.obj_mumap(datain, mMRpars, outpath=opth, store=True)

# plot

In [ ]:
# axial index
iz = 60

# plot image with a colour bar
try:  # needs HW maps
    im = muhdct['im'][iz,:,:] + muodct['im'][iz,:,:]
except:
    im = muodct['im'][iz,:,:]
plt.matshow(im, cmap='bone');

In [ ]:
hst = nipet.mmrhist(datain, mMRpars)

In [ ]:
# sinogram index (<127 for direct sinograms, >=127 for oblique sinograms)
si = 60

plt.matshow(hst['psino'][si,:,:], cmap='inferno')
plt.xlabel('bins')
plt.ylabel('angles')
plt.title('prompt sinogram')

plt.matshow(hst['dsino'][si,:,:], cmap='inferno')
plt.xlabel('bins')
plt.ylabel('angles')
plt.title('delayed sinogram');

# recon

In [ ]:
# OSEM 14 subsets
recon = nipet.mmrchain(
    datain, mMRpars,
    frames=['timings', [3000, 3600]],
    mu_h=muhdct,
    mu_o=muodct,
    itr=4,
    fwhm=0.0,
    outpath=opth,
    fcomment='niftypet-recon',
    store_img=True)

In [ ]:
plt.matshow(recon['im'][60,:,:], cmap='magma');